# DICOM, HL7 and FHIR

*Author: Bradley J. Erickson, MD PhD FSIIM*

## DICOM

DICOM stands for Digital Imaging and Communications in Medicine. It is a standard format used for storing and transmitting medical images such as X-rays, CT scans, and MRIs. DICOM files contain both image data and metadata, which includes information about the patient, the imaging equipment used, and the imaging procedure itself.
DICOM images are used in medical image processing tasks such as segmentation, registration, and classification. Segmentation is the process of separating an image into multiple regions or objects. Registration is the process of aligning two or more images of the same subject taken at different times or with different imaging modalities. Classification is the process of assigning a label to an image based on its content.
Pydicom is a Python library that provides an easy-to-use interface for working with DICOM files. It allows you to read, modify, and write DICOM files using Python code. Pydicom provides access to important information elements such as patient name, patient ID, study date, study time, modality, and more. Here’s an example code snippet that demonstrates how to use pydicom to access these information elements:


In [ ]:
# @title Run this cell to load the data and setup packages. { vertical-output: true, display-mode: "form" }
# @markdown *This might take a few seconds*.
# @markdown
# @markdown After running this cell we have:
# @markdown *   `MIDEL-CT.dcm`: a test DICOM image.
# @markdown *   `pydicom` package: used for DICOM image processing.
# @markdown *   `hl7` package: used for HL7 message parsing.

import gdown
gdown.download(
    "https://drive.google.com/uc?export=download&confirm=pbef&id=1S1zfz_UVYpf1fuzOlbiXNDpRGm6k9RPL",
    "MIDEL-CT.dcm",
    quiet=True,
)
!pip install pydicom hl7 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.2 MB/s eta 0:00:00


In [ ]:
import pydicom

# Load a DICOM file
ds = pydicom.dcmread('./MIDEL-CT.dcm')

# Access important information elements
patient_name = ds.PatientName
patient_id = ds.PatientID
study_date = ds.StudyDate
study_time = ds.StudyTime
modality = ds.Modality

# Print the information elements
print(f'Patient Name: {patient_name}')
print(f'Patient ID: {patient_id}')
print(f'Study Date: {study_date}')
print(f'Study Time: {study_time}')
print(f'Modality: {modality}')



Patient Name: MIDEL, patientname
Patient ID: MIDEL1234
Study Date: 
Study Time: 
Modality: CT


These are fairly straightforward and DICOM clearly specifies the format for representing date, time, and modality. The size and orientation of the pixels is a bit more complex--lets take a look.


In [ ]:
# Access important information elements
slice_thickness = ds.SliceThickness
pixel_spacing = ds.PixelSpacing
image_orientation = ds.ImageOrientationPatient

# Print the information elements
print(f'Slice Thickness: {slice_thickness}')
print(f'Pixel Spacing: {pixel_spacing}')
print(f'Image Orientation: {image_orientation}')


Slice Thickness: 5.000000
Pixel Spacing: [0.488281, 0.488281]
Image Orientation: [1.000000, 0.000000, 0.000000, 0.000000, 0.996917, -0.078459]


The slice thickness makes sense, and is always in millimeters, and is the distance from the ‘top’ of the slice to the ‘bottom’. Note this is **not** always the same as the slice spacing! Because DICOM is usually a one-slice at a time data object, one must do much more work to figure out slice spacing since it is the distance between two slices. While it is common for all the slices in a series to have (*nearly*) the same spacing, that is not required, and was not as common in the early days of CT imaging.

And since DICOM file names do not necessarily correlate with their position, you must go through each file and sort them into slice position order and then figure out the spacing. That sounds simple, but again is not quite so easy since the slice position is (usually) a string with a letter indicating ‘L’, ‘R’,  ‘A’, ’P’, ’S’, or ‘I’, and then a number indicating how (in millimeters) far left or right or anterior, etc., it is from isocenter.


While slice thickness refers to the edges of the voxel, the pixel dimensions refer to the spacing between pixels in the X and Y dimensions, and DICOM does expect the spacing to be constant for this image. (There are exceptions for modalities like ultrasound where there can be an 'inset' image with a different magnification).

Now that we know about the size of the pixels, what about the orientation? Many imaging devices can acquire an image in almost any plane, but CT scanners nearly always (now) acquire axial images, and other planes are reconstructed from that. MRI is more variable, and for many organs, the images are acquired in an oblique orientation.

Briefly about orientation names: ‘axial’ (also called ‘transverse’ or ‘transaxial’) means an image that is perpendicular to the long axis of the body. Furthermore, its left and right (‘X’) axis is also the left and right axis of the object being imaged, and the ‘Y’ axis of the image is considered to be in the anterior-posterior direction. A ‘Coronal’ image gets its name from the coronal suture of the skull, which is at the top of the head and is a plane that is like facing the patient: the ‘X’ axis of the image is also the ‘X’ axis of the patient, but the ‘Y’ axis of the image is the ‘Z’ axis (inferior-superior) of the patient. Sagittal gets its name from the sagittal suture, which is the connection between the left and right halves of the skull. Therefore, its ‘X’ axis is the anterior-posterior line of the patient, and its ‘Y’ axis is the  ‘Z’ axis of the patient.

<br><img src="https://i.ibb.co/gT17Qkb/orientation.jpg" alt="Figure 1" style='margin:auto' border="0"><br><u>
<b>Figure 1.</b> Different image orientations.</u><br><br>

The relationship between moving 1 pixel along the ‘X’ axis of the image is thus represented as 3 floating point values: the change in ‘X’ of the <u>patient</u>, the change in ‘Y’ of the <u>patient</u>, and the change in ‘Z’ of the <u>patient</u>. Similarly, there is another set of 3 values for showing the change as one moves along the ‘Y’ axis of the image, again representing the ‘X’, ‘Y’, and ‘Z’ change in the patient. This set of 6 values completely defines the orientation of the image, and they are known as directional cosines. In this case, the (1.0,0.0,0.0) means that the X-direction of the image moves 1.0 along the X-axis of the patient and 0 in the Y and Z axes. The second set of 3 numbers (0.0, 0.996, -0.078) means it moves 0 along the patient X axis, almost 1 (0.996) along the Y-axis, and just a little bit negative along the patient Z axis as we move along Y in the image (-0.078). From these, one can compute the orientation of the slice:


In [ ]:
import math

def getMajorAxisFromDirCos(x, y, z):
    axis = ""
    if (x < 0):
        XOrient = "R"
    else:
        XOrient = "L"
    if (y < 0):
        YOrient = "A"
    else:
        YOrient = "P"
    if (z < 0):
        ZOrient = "F"
    else:
        ZOrient = "H"

    absX = math.fabs(x)
    absY = math.fabs(y)
    absZ = math.fabs(z)

    if ((absX > 0.25) and (absX > absY) and (absX > absZ)):
        axis = XOrient
    elif ((absY > 0.25) and (absY > absX) and (absY > absZ)):
        axis = YOrient
    elif ((absZ > 0.25) and (absZ > absX) and (absZ > absY)):
        axis = ZOrient
    return axis


def getImageOrientFromDirCos(rowX, rowY, rowZ, colX, colY, colZ):
    label = ""
    rowAxis = getMajorAxisFromDirCos(rowX, rowY, rowZ)
    colAxis = getMajorAxisFromDirCos(colX, colY, colZ)

    if (rowAxis != "" and colAxis != ""):
        if ((rowAxis == "R" or rowAxis == "L") and (colAxis == "A" or colAxis == "P")):
            label = "AXL (Axial)"
        if ((rowAxis == "R" or rowAxis == "L") and (colAxis == "A" or colAxis == "P")):
            label = "AXL (Axial)"

        if ((rowAxis == "R" or rowAxis == "L") and (colAxis == "H" or colAxis == "F")):
            label = "COR (Coronal)"
        if ((rowAxis == "R" or rowAxis == "L") and (colAxis == "H" or colAxis == "F")):
            label = "COR (Coronal)"

        if ((rowAxis == "A" or rowAxis == "P") and (colAxis == "H" or colAxis == "F")):
            label = "SAG (Saggital)"
        if ((rowAxis == "A" or rowAxis == "P") and (colAxis == "H" or colAxis == "F")):
            label = "SAG (Saggital)"
    else:
        label = "OBL (Oblique)"
    return label

ImageOrientDirCos = ds.ImageOrientationPatient
Orient = getImageOrientFromDirCos(ImageOrientDirCos[0], ImageOrientDirCos[1], ImageOrientDirCos[2],
                                  ImageOrientDirCos[3], ImageOrientDirCos[4], ImageOrientDirCos[5])

print ("Image Orientation:", Orient)


Image Orientation: AXL (Axial)


DICOM is probably most similar to the TIFF image file format, in that it has many 'tags' at the start of the file that describe the image, and then the end of the file is the actual pixel information. Tags include information such as demographic details about the patient, imaging study’s acquisition parameters, image dimensions, matrix size, color space, and more. This makes it easier to manage and exchange medical images between different devices from multiple manufacturers.

There are 2 ways to identify a tag: using a pair of 4-digit hexadecimal numbers (not so user friendly) or by using text labels, where the labels come from a dictionary so you must use those exact labels. The DICOM standard group numbers (the first number of the pair) are even numbers. One can have (non-standard) tags in a DICOM file that have odd numbers. These are called 'Private Tags' and their content and meaning are defined by the creator. This allows vendors to store information relevant to an image in cases where the DICOM Committee has not defined a tag.

A commonly noted example of this is for early diffusion-weighted MRI. Vendors (and researchers) created diffusion-weighted images and the information (like B-value) was usually stored in private tags, but each vendor stored this informatoin in the private tag they defined (and this sometimes changed with different software versions). Once the community is using an image type enough, the DICOM Committee will create a working group that will define what the tags should be and they then become a part of the standard tag set. Then the vendors must update their scanner software to reflect this.

There is a HUGE amount of information about an image in the tags, particularly for CT and MR images. A great site for looking up DICOM tags is: http://dicomlookup.com/ or https://dicom.innolitics.com/.

## HL7

HL7 stands for 'Health Level Seven' where the '7' refers to an old IT concept that there are 7 layers of information systems from the low level (e.g. the packets transferred between computers) up to the highest level which is concepts. It is a set of international standards for the exchange, integration, sharing, and retrieval of electronic health information. These standards are specifically designed for the healthcare industry to facilitate the interoperability of health information systems. HL7 standards are widely used to enable the seamless communication and data exchange between various healthcare systems, including electronic health records (EHRs), laboratory information systems, pharmacy systems, and more.

A few key aspects to remeber about HL7:

**Versions:** HL7 defines messaging standards, such as HL7 Version 2 (HL7 v2) and HL7 Version 3 (HL7 v3), which specify the format and content of messages exchanged between healthcare applications. HL7 v2 is widely used for its simplicity and widespread adoption, while HL7 v3 offers more semantic interoperability but is less commonly used.

**Clinical Document Architecture (CDA):** HL7 CDA is a standard for creating and sharing clinical documents, such as discharge summaries and progress notes, in a structured format. CDA documents are XML-based and can be easily exchanged between different EHR systems.

**Terminology Standards:** HL7 incorporates standard terminologies like SNOMED CT, LOINC, and RxNorm to ensure consistency in coding and describing clinical data. This helps in semantic interoperability and accurate data exchange.

**Integration with Healthcare IT Systems:** HL7 standards are used for integrating various healthcare IT systems, including EHRs, laboratory systems, radiology information systems, and more. This integration helps healthcare organizations streamline workflows and improve patient care.



### HL7 Message Types

HL7 defines several types of messages for exchanging healthcare information. These messages serve different purposes and are used in various healthcare scenarios. Some of the most common types of HL7 messages include:

1. **ADT (Admit, Discharge, Transfer) Messages (HL7 v2.3 and later):** These messages are used for patient demographic and visit information, including admissions, discharges, and transfers. They provide details about a patient's admission to or discharge from a healthcare facility and any changes in their demographic information.

2. **ORM (Order Message) and ORU (Observation Result Message) Messages (HL7 v2.3 and later):** ORM messages are used to transmit orders for laboratory tests, procedures, or medications. ORU messages, on the other hand, convey the results of these orders, such as lab results, diagnostic reports, or imaging studies.

3. **SIU (Scheduling Information Unsolicited) Messages (HL7 v2.3 and later):** These messages contain unsolicited scheduling information, such as appointment notifications, cancellations, and rescheduling updates.

4. **MDM (Medical Document Management) Messages (HL7 v2.3 and later):** MDM messages are used for the exchange of medical documents, including clinical reports, discharge summaries, and other clinical documents.

5. **ADT^Axx (ADT Acknowledgment) Messages (HL7 v2.3 and later):** These acknowledgment messages are sent in response to ADT messages and provide confirmation or notification of errors or exceptions.

6. **QRY (Query) Messages (HL7 v2.3 and later):** QRY messages are used for querying patient data or other information from a remote system. They can be used to request patient demographic information, clinical data, or other relevant data.

7. **RSP (Response) Messages (HL7 v2.3 and later):** RSP messages are sent in response to QRY messages and contain the requested data or information.

8. **MFN (Master File Notification) Messages (HL7 v2.3 and later):** These messages are used to update master files, such as patient or provider records, by notifying other systems of changes or updates.

9. **BAR (Billing Account Record) Messages (HL7 v2.3 and later):** BAR messages are used for financial transactions and billing-related information, such as insurance claims and invoices.

10. **MFQ (Master File Query) Messages (HL7 v2.3 and later):** These messages are used to query master files, such as patient or provider records, to retrieve specific information.

11. **MFQ/MFR (Master File Query/Response) Messages (HL7 v2.5 and later):** MFQ/MFR messages combine querying and responding to master file information in a single message exchange.

12. **VXU (Vaccination Update) Messages (HL7 v2.3.1 and later):** VXU messages are used for the exchange of vaccination and immunization data, including patient vaccination histories.

13. **PPR (Patient Problem) Messages (HL7 v2.5 and later):** PPR messages convey information about patient problems, such as medical conditions, allergies, and other health concerns.

These are some of the key HL7 message types used in healthcare interoperability. The choice of message type depends on the specific information being exchanged and the purpose of the communication within the healthcare system. Different versions of HL7 (e.g., HL7 v2.x, HL7 v3, and HL7 FHIR) have variations in message types and structures to accommodate evolving healthcare needs and technology advancements.

### HL7 Message Segements

HL7 message segments are fundamental building blocks of HL7 messages. These segments serve as containers for specific types of information within an HL7 message. Each segment has a predefined structure and a two-to-four-letter code that identifies its purpose and the type of data it contains. Here are some common HL7 message segments:

1. **MSH (Message Header):** The MSH segment is the first segment in an HL7 message. It contains information about the message itself, such as the message type, source, destination, and timestamps. It helps in routing and processing the message correctly.

2. **PID (Patient Identification):** The PID segment contains patient demographic information, including patient ID, name, date of birth, sex, and other patient-specific details. It is essential for accurately identifying and managing patient records.

3. **PV1 (Patient Visit Information):** The PV1 segment provides information about the patient's visit to a healthcare facility. It includes data such as admission and discharge dates, visit number, patient class, and attending physician.

4. **ORC (Common Order):** The ORC segment is used in order-related messages (e.g., ORM) and contains information about orders, including order control codes, ordering provider, and order status.

5. **OBR (Observation Request):** The OBR segment is used to request clinical observations or tests. It includes details such as the test name, specimen source, and relevant clinical information.

6. **OBX (Observation Result):** The OBX segment carries the results of clinical observations or tests requested in the OBR segment. It includes the observed value, units of measurement, reference ranges, and interpretation codes.

7. **NK1 (Next of Kin):** The NK1 segment contains information about the patient's next of kin or emergency contacts, including their names, relationships, and contact details.

8. **DG1 (Diagnosis):** The DG1 segment is used to convey diagnostic information, such as patient diagnoses, admitting diagnoses, and diagnosis codes (e.g., ICD-10 codes).

9. **RXA (Pharmacy/Treatment Administration):** The RXA segment is used in medication administration messages (e.g., ADT) to convey details about medication administration, including drug name, dose, route, and administration date/time.

10. **GT1 (Guarantor):** The GT1 segment contains information about the patient's financial guarantor or responsible party, including their names, addresses, and insurance information.

11. **IN1 (Insurance):** The IN1 segment includes details about the patient's insurance coverage, such as insurance plan name, policy number, and eligibility information.

12. **Z-Segments (User-Defined Segments):** Z-segments are user-defined segments that allow for the inclusion of custom or site-specific data within an HL7 message. Healthcare organizations can use these segments to exchange additional information specific to their needs. These are analogous to DICOM's private tags.

This is one example of an ORM HL7 message:

```
MSH|^~&|HIS|MIDELMedCenter|RIS|MIDELMedCenter|20160307110114||ORM^O01|MSGID20060307110114|P|2.3\r
PID|||12001||Smith^John^^^Mr.||19570824|M|||123 West St.^^Minneapolis^MN^55431^USA|||||||\r
PV1||O|OP^PAREG^||||2342^Johnson^Bob|||OP|||||||||2|||||||||||||||||||||||||20160307110111|\r
ORC|NW|20160307110114\r
OBR|1|20160307110114||003038^Chest Radiograph^L|||20160307110114\r
```

Let's parse this message:

In [ ]:
import hl7

# HL7 message as a string
message = "MSH|^~&|HIS|MIDELMedCenter|RIS|MIDELMedCenter|20160307110114||ORM^O01|MSGID20060307110114|P|2.3\r" + \
          "PID|||12001||Smith^John^^^Mr.||19570824|M|||123 West St.^^Minneapolis^MN^55431^USA|||||||\r" +  \
          "PV1||O|OP^PAREG^||||2342^Johnson^Bob|||OP|||||||||2|||||||||||||||||||||||||20160307110111|\r" + \
          "ORC|NW|20160307110114\r" + \
          "OBR|1|20160307110114||003038^Chest Radiograph^L|||20160307110114\r"

parsed_message = hl7.parse(message)

print("Number of segments:",len(parsed_message))
print("OBR segment is parsed as:")
print(parsed_message.segments("OBR"))

Number of segments: 5
OBR segment is parsed as:
[[['OBR'], ['1'], ['20160307110114'], [''], [[['003038'], ['Chest Radiograph'], ['L']]], [''], [''], ['20160307110114']]]


As you see, there is no easy way to understand what each value represents. These are just some examples of HL7 message segments. The choice of segments used in an HL7 message depends on the message type and the specific information being exchanged. Healthcare informatics professionals need to be familiar with the structure and purpose of these segments to effectively process and interpret HL7 messages, ensuring accurate and meaningful healthcare data exchange.

## Fast Healthcare Interoperability Resources (FHIR)

FHIR is a modern healthcare interoperability standard developed by Health Level Seven International (HL7) starting in 2016, due to challenges around HL7, particuarly querying information, but also the desire to have a protocol using modern IT technologies. FHIR is designed to address the challenges of exchanging healthcare data in a more efficient, flexible, and standardized manner. It has gained significant attention and adoption within the healthcare informatics field due to its innovative approach.

FHIR is based on modern web standards like RESTful APIs (Representational State Transfer) and uses widely adopted data formats like JSON and XML for data exchange. This approach makes it developer-friendly and accessible for software applications and web services. It follows the "resource" model, where healthcare data is represented as resources with standardized data elements and relationships. Resources can represent various healthcare entities, such as patients, providers, medications, and observations.
FHIR emphasizes granular data exchange. Instead of sending large, complex documents, it allows the exchange of individual data elements or resources. This granularity enables precise data retrieval and efficient updates.

FHIR defines a set of standardized resources, each representing a specific aspect of healthcare data. For example, the "Patient" resource contains demographic information, while the "Observation" resource represents clinical observations like lab results or vital signs. Each resource has a well-defined structure with common data elements (attributes) and relationships to other resources.

FHIR allows for extensibility by enabling the addition of custom data elements or extensions to existing resources. This flexibility accommodates local or specialized data requirements. This can lead to challenges if you don't know the definitions of these added messages.

For example, for the previous HL7 message, the OBR segment can be reorderd as the following FHIR message:

```json
{
  "resourceType": "DiagnosticReport",
  "id": "20160307110114",
  "status": "final",
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "003038",
        "display": "Chest Radiograph"
      }
    ]
  },
  "subject": {
    "reference": "Patient/12001"
  },
  "effectiveDateTime": "2016-03-07T11:01:14",
  "issued": "2016-03-07T11:01:14"
}
```

The FHIR implementation guides provide detailed specifications and profiles for specific use cases, specialties, or regions. These guides help standardize data exchange in specialized contexts.

In summary, FHIR is a groundbreaking standard in healthcare informatics that addresses the need for flexible, efficient, and standardized data exchange in the digital healthcare landscape. Its adoption continues to grow, and it plays a crucial role in achieving healthcare interoperability, supporting innovation in healthcare applications, and improving patient care.